In [1]:
from qem.crystal_analyzer import CrystalAnalyzer
import hyperspy.api as hs   
import scipy.io as sio

file = '/home/zzhang/OneDrive/data/Yang Zhang STO/dynamic pc AXON/adf_avg_25.hspy'
s = hs.load(file)
image = s.data
dx =0.071356
matfile_path = '/home/zzhang/OneDrive/data/Yang Zhang STO/dynamic pc AXON/adf_avg_coords.mat'
matfile = sio.loadmat(matfile_path)
coords = matfile['coords'].astype(float)
crystal_analyzer = CrystalAnalyzer(image, dx, coords[:,:2]/dx,coords[:,2].astype(int)-1,['Y','Al','O'])

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:hyperspy.extensions:Enabling extension exspy
INFO:hyperspy.defaults_parser:Writing the config file


In [2]:
%matplotlib qt
import matplotlib.pyplot as plt
crystal_analyzer.plot()

/home/zzhang/OneDrive/code/qem/qem/crystal_analyzer.py:64: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  plt.scatter(


In [3]:
import numpy as np
cif_file_path = '/home/zzhang/OneDrive/data/Yang Zhang STO/STEM/72104-CaYAlO4.cif'

crystal_analyzer.import_crystal_structure(cif_file_path)
crystal_analyzer.transform(np.array([[1,0,0],[0,0,1],[0,1,0]]))


In [4]:
import numpy as np
crystal_analyzer.origin = np.array([831.26558017, 962.21710935, 0])
crystal_analyzer.a = np.array([ 31.26403339, -36.73158211,0])
crystal_analyzer.b = np.array([-118.32881689, -100.67459688,0])
crystal_analyzer.c = np.array([  0.        ,   0.        , crystal_analyzer.unitcell.lattice.abc[2]])


In [5]:
# %matplotlib qt
# crystal_analyzer.choose_lattice_vectors()

In [6]:
crystal_analyzer.unitcell_mapping()

array([[ -75.94816783,  -64.61698326,    0.        ,    0.        ],
       [ -42.38064906,  -36.05761362,    0.        ,    0.        ],
       [  -1.15174269,  -32.64547588,    1.82255   ,    0.        ],
       [ -85.91304081, -104.76070311,    1.82255   ,    0.        ],
       [ -43.53239175,  -68.70308949,    1.82255   ,    1.        ],
       [   0.        ,    0.        ,    0.        ,    1.        ],
       [ -27.90037505,  -87.06888055,    1.82255   ,    2.        ],
       [ -43.53239175,  -68.70308949,    0.        ,    2.        ],
       [  15.6320167 ,  -18.36579105,    0.        ,    2.        ],
       [  31.26403339,  -36.73158211,    1.82255   ,    2.        ],
       [ -98.46140853,  -83.77133206,    0.        ,    2.        ],
       [ -19.86740836,  -16.90326482,    0.        ,    2.        ],
       [ -23.66498339,  -51.79982468,    1.82255   ,    2.        ],
       [ -63.39980011,  -85.60635431,    1.82255   ,    2.        ]])

In [6]:
crystal_analyzer.supercell_with_refined_peaks(a_limit=20,b_limit=15)
crystal_analyzer.write_lammps()

In [9]:
crystal_analyzer.min_distances()

{'Ca': 20.68101144143594,
 'Y': 20.68101144143594,
 'Al': 40.66688959593986,
 'O': 12.058837288775779}

In [8]:
crystal_analyzer.write_xyz()